## Setting up feasible, stable systems

In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from numpy import linalg
from scipy.integrate import solve_ivp
import os

In [2]:
# GLV parameter creation
#Function that finds the maximum eigenvalue
def max_eigenval(A, equilibrium):
    N = len(equilibrium)
    M = np.zeros([N,N])
    for i in range(N):
        for j in range(N):
            M[i][j]=equilibrium[i]*A[i][j]
        
    eigenvals = linalg.eig(M)[0]
    return np.max(eigenvals.real)

#Function to build May type matrices. Amount of zero elements can be set by C
def build_May_normal(n, C, d, sigma):
    
    # fill the whole matrix
    M = np.random.normal(loc=0, scale=sigma, size=(n, n))
    #M = np.random.normal(loc=np.random.uniform(-3,2), scale=np.random.uniform(0.01,1), size=(n,n))
    
    # remove connections
    M *= np.random.uniform(size=(n, n)) <= 1-C
    
    # set diagonals
    np.fill_diagonal(M, d)
    #diagonal = np.random.uniform(d,-d, size= n)
    #np.fill_diagonal(M,diagonal)
    return np.array(M)

# Produce a large arbitrary equilibrium point vector to be used in the simulation
equilibrium_seed = 172
np.random.seed(equilibrium_seed)
N = 12
equilibrium = np.abs(np.random.normal(10, 0.5, size=N))
#equilibrium = np.full((N), 10.0) 
print(equilibrium)

[10.57804896  9.40620747 10.46450969 10.13572668  9.45096018  9.41139386
  9.85162229  9.17873928 10.55409368 10.4724744  10.21967809  9.55966778]


In [3]:
# Find a suitable seed number for producing interaction matrix A for the above equilibrium
#Parameters used to produce matrices
C = 0.0
#np.random.seed(1)
#d = np.random.normal(loc = -4.4, scale = 1, size=N)
d = -np.sqrt(N)
sig = 1

for s in range(100):
    np.random.seed(s)
    A = build_May_normal(N, C, d, sig)
    #A = build_May_normal_modified(N, C, sig)
    r = -np.matmul(A, equilibrium)
    maximum_eigenvalue = max_eigenval(A, equilibrium)
    if max_eigenval(A, equilibrium)<0:
        print(maximum_eigenvalue, s, linalg.cond(A))

-13.153065406505215 0 9.626352644879919
-8.173012987243306 1 12.987851773697376
-12.360374864319464 2 13.75924057338846
-5.19723188355548 3 23.399880649896286
-5.683708572367714 4 15.599570269110623
-8.85355930862486 5 11.710111351436563
-5.918622072912666 6 39.17968110863993
-4.585040109107911 8 14.525790003758997
-14.289571869136049 11 8.784235338995302
-5.314837684066439 12 21.71197069257384
-13.101177913835157 13 7.837643594043283
-9.677617034325063 14 14.122961071039361
-14.504483490611628 15 8.157672771980964
-8.248707990213365 16 7.700886829252556
-1.0976586632613898 17 122.10810883323629
-9.285380364748317 19 8.010862835171864
-10.95390948832568 20 35.957066102143806
-3.412666253487361 21 37.708533555906676
-2.2522632002373375 22 37.12524194497604
-4.244251003044462 23 22.310522857720095
-20.22241685965052 24 6.537445320005202
-14.194107265131972 25 10.237713094007253
-3.419631279486022 26 28.41800551454336
-10.06431362755565 27 4.819639614719018
-11.405750459461045 28 7.576899

In [4]:
# Set the variables from one of the above seeds
matrix_seed = 5
np.random.seed(matrix_seed)
# Define parameters
A = build_May_normal(N, C, d, sig)

with open('parameters.txt', 'w') as f:
    f.write(f'N = {N}\n')
    f.write('A = \n')
    np.savetxt(f, A, fmt='%.8f')
    f.write('equilibrium = \n')
    np.savetxt(f, equilibrium, fmt='%.8f')